<a href="https://colab.research.google.com/github/stiwen87/adComOfflineCalc/blob/main/offlineCalc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
def translateNum(num):
    res = [0]*2
    # split na '21.58' i 'ee'
    num = num.replace(',','.')
    tmp = num.split()
    # 21.58ee -> [2.158, 28]
#m6 b9 t12 a15 b18 c21 d24 e27 f30 g33 h36 i39 j42 k45 l48 m51 n54 o57 p60 q63 r66 s69 t72 u75 v78
    values = {'': 0, 'K': 3, 'M': 6, 'B': 9, 'T': 12, 'AA': 15, 'BB': 18, 'CC': 21, 'DD': 24, 'EE': 27, 'FF': 30,
              'GG': 33, 'HH': 36, 'II': 39, 'JJ': 42, 'KK': 45, 'LL': 48, 'MM': 51, 'NN': 54, 'OO': 57,
              'PP': 60, 'QQ': 63, 'RR': 66, 'SS': 69, 'TT': 72, 'UU': 75, 'VV': 78, 'WW': 81, 'XX': 84,
              'YY': 87, 'ZZ': 90, 'AAA': 93}
    if len(tmp) > 1:
        res[1] = values.get(tmp[1].upper())
    res[0] = float(tmp[0])

    return normalizeNum(res)

def reverseTranslate(num):
    values = {'': 0, 'K': 3, 'M': 6, 'B': 9, 'T': 12, 'AA': 15, 'BB': 18, 'CC': 21, 'DD': 24, 'EE': 27, 'FF': 30,
              'GG': 33, 'HH': 36, 'II': 39, 'JJ': 42, 'KK': 45, 'LL': 48, 'MM': 51, 'NN': 54, 'OO': 57,
              'PP': 60, 'QQ': 63, 'RR': 66, 'SS': 69, 'TT': 72, 'UU': 75, 'VV': 78, 'WW': 81, 'XX': 84,
              'YY': 87, 'ZZ': 90, 'AAA': 93}

    return "{:.2f}".format(num[0]*(10**(num[1]%3)))+' '+list(values.keys())[list(values.values()).index(num[1]//3*3)]

def normalizeNum(num):
    if num[0] == 0:
        return num

    while num[0] >= 10:
        num[1] += 1
        num[0]/=10

    while num[0] < 1:
        num[1] -= 1
        num[0]*=10

    return num

def addNums(numA, numB):
    if numB[1] > numA[1]:
        numA, numB = numB, numA

    diff = numA[1]-numB[1]
    if diff > 2:
        return numA
    res = [numA[0]*10**diff + numB[0], numB[1]]
    return normalizeNum(res)

def divideNum(numA, numB):
    res = [numA[0]/numB[0], numA[1]-numB[1]]
    return normalizeNum(res)

def minNum(numA, numB):
    normalizeNum(numA)
    normalizeNum(numB)
    if numA[1] == numB[1]:
        return numA if numA[0] < numB[0] else numB
    return numA if numA[1] < numB[1] else numB

def maxNum(numA, numB):
    normalizeNum(numA)
    normalizeNum(numB)
    if numA[1] == numB[1]:
        return numA if numA[0] > numB[0] else numB
    return numA if numA[1] > numB[1] else numB

def calcOffline(generators, realOutput, costs, time, actions, buyLimit):
    #rnnlb
    n = len(generators)
    res = [[0]*2 for _ in range(n+1)]
    buy = False

    for i in range(n, -1, -1):
        if actions[i] == 's':
            continue
        elif actions[i] == 'r' or actions[i] == 'n':
            res[i][0] = 3600*time* res[i+1][0]*realOutput[i]
            res[i][1] = res[i+1][1]
            normalizeNum(res[i])

        elif actions[i] == 'l':
            res[i][0] = 3600*time* generators[i][0]*realOutput[i]
            res[i][1] = generators[i][1]
            normalizeNum(res[i])
        elif actions[i] == 'b':
           buy = True
        if actions[i] == 'n' or actions[i] == 'l':
                res[i] = addNums(res[i], generators[i-1])

    if buy:
        res[n] = divideNum(res[0], costs[n-1])
        res[n] = addNums(res[n], generators[n-1])

        if buyLimit != -1:
            limit = translateNum(buyLimit)
            res[n] = minNum(res[n], limit)
    else:
        res[n] = generators[n-1]

    return res


#generators costs
rawCosts1 = ['10', '1000', '100 b', '10 cc', '100 ff', '10 ii']
rawCosts2 = ['25', '100 m', '10 bb', '10 ee', '100 hh']
rawCosts3 = ['50', '100 b', '1 cc', '10 ff']
costs1 = list(map(translateNum,rawCosts1))
costs2 = list(map(translateNum,rawCosts2))
costs3 = list(map(translateNum,rawCosts3))

basicOutput1 = [2, 1, .444, .0185, .0741, .0288]
basicOutput2 = [2, 1, .375, .125, .0391]
basicOutput3 = [2, .75, .194, .044]

basicCommonsPower1 = [1.5, 2, 2.5, 3, 3.5, 4]
basicCommonsPower2 = [2.5, 3, 3.5, 4, 4.5]
basicCommonsPower3 = [3.5, 4, 4.5, 5]
basicRarePowers = [2, 3, 4]



#edit diz
#card lvls
commons1 = [7, 7, 7, 7, 7, 7]
commons2 = [7, 7, 7, 6, 5]
commons3 = [7, 6, 6, 6]
rares = [5, 5, 4]

#generators amount
rawGen1 = ['111.13 kk', '530.50 hh', '4.50 ff', '73,75 cc', '4.84 aa', '832.02 m']
rawGen2 = ['883 hh', '1.95 ff', '9.98 cc', '283 t', '50 m']
rawGen3 = ['17.7 ee', '95.8 bb', '2.83 t', '1.01 m']



generators1 = list(map(translateNum, rawGen1))
generators2 = list(map(translateNum, rawGen2))
generators3 = list(map(translateNum, rawGen3))

realRarePowers = list(map(lambda x, y: y*(2**x) if x > 0 else 1, rares, basicRarePowers))

realOutput1 = list(map(lambda x,y,z: x*y*(2**z)*realRarePowers[0] if z > 0 else 0,
                       basicOutput1, basicCommonsPower1, commons1))
realOutput2 = list(map(lambda x,y,z: x*y*(2**z)*realRarePowers[1] if z > 0 else 0,
                       basicOutput2, basicCommonsPower2, commons2))
realOutput3 = list(map(lambda x,y,z: x*y*(2**z)*realRarePowers[2] if z > 0 else 0,
                       basicOutput3, basicCommonsPower3, commons3))


#print(translateNum('530,5 hh'))

res = translateNum('0')
maxI, maxJ = 0, 0

after = None

time = 10
for i in range(time):
    after = calcOffline(generators3, realOutput3, costs3, i, 'rnnlb', -1)
    for j in range(time-i):
        after = calcOffline(after[1:], realOutput3, costs3, j, 'rnnlb', -1)
        after = calcOffline(after[1:], realOutput3, costs3, time-i-j, 'rnnlb', -1)
        tmp = maxNum(res, after[-1])
        if tmp != res:
            maxI, maxJ = i, j
            res = tmp

print(maxI, maxJ)

after = calcOffline(generators3, realOutput3, costs3, 1, 'rnnlb', '50 m')
after = calcOffline(after[1:], realOutput3, costs3, 1, 'rnnlb', '50 m')
print(list(map(reverseTranslate, after)))

0 9
['8.14 HH', '39.44 EE', '317.10 BB', '11.08 T', '2.35 M']


In [4]:
import copy


unlocked = [[0]*4 for _ in range(31)]
unlo1 = [1,3,4,7,10,14,22]
unlo2 = [5,7,11,15,21,24]
unlo3 = [8,12,17,21,26]
unlo4 = [15,19,24,28]

unlockedRares = [[0]*4 for _ in range(31)]
rareProd = [1,4,7,10,13,20,23]
rareTrad = [1,6,11,16,18]

for i in range(31):
    c1, c2, c3, c4, rp, rt = 0,0,0,0,0,0
    while c1<len(unlo1) and i >= unlo1[c1]:
        c1+=1
    while c2<len(unlo2) and i >= unlo2[c2]:
        c2+=1
    while c3<len(unlo3) and i >= unlo3[c3]:
        c3+=1
    while c4<len(unlo4) and i >= unlo4[c4]:
        c4+=1
    while rp<len(rareProd) and i >= rareProd[rp]:
        rp+=1
    while rt<len(rareTrad) and i >= rareTrad[rt]:
        rt+=1
    unlocked[i] = [c1,c2,c3,c4]
    unlockedRares[i] = [rp, rt]
print(unlockedRares)

cards = [0,24,21,56,43,72,93,190,110,120,144,123,256,207,324,158,
         323,294,396,368,384,297,500,357,520,424,594,440,616,513,754]

perlevel = [1, 5, 15, 35, 85, 185, 385, 785]

res = [[] for _ in range(31)]
res[0] = [[],[],[],[]]
resLvl = [[] for _ in range(31)]

resR = [[] for _ in range(31)]
resR[0] = [[],[]]
resRLvl = [[] for _ in range(31)]


#[[0],[0],[0],[0]]
#[[22],[0],[0],[0]]

for i in range(1,31):
    sum,sumR = 0,0
    for j in range(len(unlocked[i])):
        sum+= unlocked[i][j]
    for j in range(len(unlockedRares[i])):
        sumR+= unlockedRares[i][j]
    tmp = cards[i]*0.9/sum
    tmpR = cards[i]*0.1/sumR
    res[i] = copy.deepcopy(res[i-1])
    for j in range(len(unlocked[i])):
        while len(res[i][j])<unlocked[i][j]:
            res[i][j].append(0)
        for k in range(len(res[i][j])):
            res[i][j][k]+= tmp
    resLvl[i] = copy.deepcopy(res[i])
    for j in range(len(unlocked[i])):
        for k in range(len(res[i][j])):
            resLvl[i][j][k]=0
            while resLvl[i][j][k]<len(perlevel) and res[i][j][k]>perlevel[resLvl[i][j][k]]:
                resLvl[i][j][k]+= 1

    resR[i] = copy.deepcopy(resR[i-1])
    for j in range(len(unlockedRares[i])):
        while len(resR[i][j])<unlockedRares[i][j]:
            resR[i][j].append(0)
        for k in range(len(resR[i][j])):
            resR[i][j][k]+= tmpR
    resRLvl[i] = copy.deepcopy(resR[i])
    for j in range(len(unlockedRares[i])):
        for k in range(len(resR[i][j])):
            resRLvl[i][j][k]=0
            while resRLvl[i][j][k]<len(perlevel) and resR[i][j][k]>perlevel[resRLvl[i][j][k]]:
                resRLvl[i][j][k]+= 1

prodNames = ['heist', 'thing', 'crit', 'witch', 'ryder', 'elfs', 'utars']
tradeNames = ['mama', 'ben10', 'i3', 'frosty', 'i4']

print(res[10])
for i in range(1,31):
    print(i, end="")
    print(": ", end="")
    print(resLvl[i], end="")
    print("; R: ", end="")
    for j in range(len(resRLvl[i][0])):
        print(prodNames[j], end='')
        print(': ', end="")
        print(resRLvl[i][0][j], end="")
        print(', ',end="")
    for j in range(len(resRLvl[i][1])):
        print(tradeNames[j], end='')
        print(': ', end="")
        print(resRLvl[i][1][j], end="")
        print(', ',end="")
    print()
    print(resRLvl[i])

# rareProd = [1,4,7,10,13,20,23]
# rareTrad = [1,6,11,16,18]
print("-------------")

for i in range(31):
    print(i,end="")
    print(": ", end="")
    print(resR[i])

[[0, 0], [1, 1], [1, 1], [1, 1], [2, 1], [2, 1], [2, 2], [3, 2], [3, 2], [3, 2], [4, 2], [4, 3], [4, 3], [5, 3], [5, 3], [5, 3], [5, 4], [5, 4], [5, 5], [5, 5], [6, 5], [6, 5], [6, 5], [7, 5], [7, 5], [7, 5], [7, 5], [7, 5], [7, 5], [7, 5], [7, 5]]
[[189.99642857142857, 149.49642857142854, 124.29642857142858, 74.27142857142857, 16.2], [111.39642857142857, 74.27142857142857], [45.77142857142857], []]
1: [[3], [], [], []]; R: heist: 1, mama: 1, 
[[1], [1]]
2: [[4], [], [], []]; R: heist: 1, mama: 1, 
[[1], [1]]
3: [[4, 3], [], [], []]; R: heist: 2, mama: 2, 
[[2], [2]]
4: [[4, 4, 2], [], [], []]; R: heist: 2, thing: 1, mama: 2, 
[[2, 1], [2]]
5: [[5, 4, 3], [3], [], []]; R: heist: 2, thing: 1, mama: 2, 
[[2, 1], [2]]
6: [[5, 4, 4], [4], [], []]; R: heist: 2, thing: 2, mama: 2, ben10: 1, 
[[2, 2], [2, 1]]
7: [[5, 5, 4, 3], [4, 3], [], []]; R: heist: 3, thing: 2, crit: 1, mama: 3, ben10: 2, 
[[3, 2, 1], [3, 2]]
8: [[5, 5, 5, 4], [4, 4], [2], []]; R: heist: 3, thing: 2, crit: 2, mama: 3, be